In [3]:
!git clone https://github.com/ferdinandl007/Northern-Data-Star-Wars-GAN.git
%cd /content/Northern-Data-Star-Wars-GAN/DatasetGenerator
!pip install --upgrade pip
!pip install -r requirements.txt

Cloning into 'Northern-Data-Star-Wars-GAN'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 115 (delta 3), reused 7 (delta 3), pack-reused 104
Receiving objects: 100% (115/115), 103.98 MiB | 52.50 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/Northern-Data-Star-Wars-GAN/DatasetGenerator


Run this if you want to generate the dataset from scratch when generating the dataset like this it will automatically download the videos and extract faces additionally you can define your own videos to be used to generate the training said when adding the video you're well parameter like this ''''GenerateDataset(every_x_frame=13, video_urls= [])''''

In [ ]:
from GenerateDataset import GenerateDataset
dataset_generator = GenerateDataset(every_x_frame=13, set_fix_image_size=256)
dataset_generator.run()


Alternatively download it here!

In [4]:
%cd /content/Northern-Data-Star-Wars-GAN/DatasetGenerator/
!wget https://www.dropbox.com/s/v6ffbfof26byuy1/faces.zip
from zipfile import ZipFile
zf = ZipFile('faces.zip', 'r')
zf.extractall()
zf.close()


/content/Northern-Data-Star-Wars-GAN/DatasetGenerator
--2021-02-16 11:15:41--  https://www.dropbox.com/s/xaeeuut48ujkoi2/faces.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xaeeuut48ujkoi2/faces.zip [following]
--2021-02-16 11:15:42--  https://www.dropbox.com/s/raw/xaeeuut48ujkoi2/faces.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc786ebbe5ff094c784ec6a1b7c9.dl.dropboxusercontent.com/cd/0/inline/BJB_zSUYpa4rTvcVbb0x2k3vah0Vs-uaR7kqSf00u8koz5FzStVCkFZa4mVeDnVupUtBZSI-ECHjtg6Azm3Yiz_B1lm86Y93pi3GFI4j1wc2dnt5eADNZ936kWw8EFM3O7E/file# [following]
--2021-02-16 11:15:43--  https://uc786ebbe5ff094c784ec6a1b7c9.dl.dropboxusercontent.com/cd/0/inline/BJB_zSUYpa4rTvcVbb0x2k3vah0Vs-uaR7kqSf00u8koz5FzStVCkFZa4mVeDnVu

In [ ]:
%cd /content/Northern-Data-Star-Wars-GAN/DatasetGenerator/faces
!python /content/Northern-Data-Star-Wars-GAN/stylegan2/dataset_tool.py create_from_images_raw /content/Northern-Data-Star-Wars-GAN/stylegan2/dataset/starWars .

In [ ]:
%cd /content/Northern-Data-Star-Wars-GAN/stylegan2
!python run_training.py --num-gpus=8 --data-dir=./dataset --config=config-e --dataset=starWars --min-h=3 --min-w=3 --res-log2=7 --mirror-augment=true --metric=none --total-kimg=20000 --minibatch-gpu-base=1 --result-dir="/content/Northern-Data-Star-Wars-GAN/results"

In [ ]:
%cd /content/Northern-Data-Star-Wars-GAN/stylegan2
!wget https://hanlab.mit.edu/projects/data-efficient-gans/models/stylegan2-ffhq.pkl

In [ ]:
%cd /content/Northern-Data-Star-Wars-GAN/stylegan2
from IPython.display import Image 
import blend_models
resolutions = (8, 16, 32, 64, 128)
for res in resolutions:
  filename = f"blended-{res}.jpg"
  blend_models.main("starWars.pkl", "stylegan2-ffhq.pkl", res, output_grid=filename)
  img = Image(filename=filename)
  print(f"blending at {res}x{res}")
  display(img)

In [ ]:
!python blend_models starWars.pkl stylegan2-ffhq.pkl 32 --output-pkl="blended.pkl"

In [ ]:
!mkdir aligned
!mkdir generate
!mkdir row

In [ ]:
# import pretrained_networks
# _, _, Gs_blended = pretrained_networks.load_networks()
# _, _, Gs = pretrained_networks.load_networks(ffhq_url)

In [ ]:
!python align_images.py raw aligned

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://127.0.0.1:8890/
Error: Kernel Python 3 is not usable. Check the Jupyter output tab for more information.

In [ ]:
!python project_images.py --num-steps 500 aligned generated

In [ ]:
import numpy as np
from PIL import Image
import dnnlib
import dnnlib.tflib as tflib
from pathlib import Path

latent_dir = Path("generated")
latents = latent_dir.glob("*.npy")
for latent_file in latents:
  latent = np.load(latent_file)
  latent = np.expand_dims(latent,axis=0)
  synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
  images = Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs)
  Image.fromarray(images.transpose((0,2,3,1))[0], 'RGB').save(latent_file.parent / (f"{latent_file.stem}-toon.jpg"))

In [ ]:
from IPython.display import Image 
embedded = Image(filename="generated/example_01.png", width=256)
display(embedded)
tooned = Image(filename="generated/example_01-toon.jpg", width=256)
display(tooned)